# 工具  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
工具允许智能体在现实世界中"行动"。
仔细的描述可以帮助你的智能体发现如何使用你的工具。

LangChain 支持多种工具格式和工具集。这里我们将介绍一些常见情况，但请查看[文档](https://docs.langchain.com/oss/python/langchain/tools)以获取更多信息。

## 设置

加载和/或检查所需的环境变量

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")

## 计算器示例

在此示例中，LLM 使用文档字符串和推断的参数及参数类型来确定何时以及如何调用工具。

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """对两个实数执行基本算术运算。"""
    print("🧮 调用计算器工具")
    # 执行指定的运算
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("不允许除以零。")
        return a / b
    else:
        raise ValueError(f"无效的运算：{operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

这会调用你的计算器工具。

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

我们可以在 [LangSmith 可观测性](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r)中检查**元数据**来查看这一点。

工具描述可能会产生很大影响。
这可能不会调用你的计算器工具，因为输入是整数。（结果因运行而异）

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

尽管输入是实数，但通常不会调用工具。（结果因运行而异）

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

## 添加更详细的描述
虽然基本描述通常就足够了，但 LangChain 支持增强描述。下面的示例使用了一种方法：Google 风格的参数描述。与 `parse_docstring=True` 一起使用时，这将解析参数描述并将其传递给模型。你可以重命名工具并更改其描述。当你共享标准工具但希望提供特定于智能体的说明时，这会很有效。

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "对两个实数执行基本算术运算。"
        "每当你对任何数字进行运算时都使用此工具，即使它们是整数。"
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """对两个实数执行基本算术运算。

    Args:
        a (float): 第一个数字。
        b (float): 第二个数字。
        operation (Literal["add", "subtract", "multiply", "divide"]):
            要执行的算术运算。

            - `"add"`: 返回 `a` 和 `b` 的和。
            - `"subtract"`: 返回 `a - b` 的结果。
            - `"multiply"`: 返回 `a` 和 `b` 的乘积。
            - `"divide"`: 返回 `a / b` 的结果。如果 `b` 为零，则引发错误。

    Returns:
        float: 指定运算的数值结果。

    Raises:
        ValueError: 如果提供了无效的运算或尝试除以零。
    """
    print("🧮  调用计算器工具")
    # 执行指定的运算
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("不允许除以零。")
        return a / b
    else:
        raise ValueError(f"无效的运算：{operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

让我们检查我们的 [LangSmith 可观测性追踪](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r)以查看工具描述。

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

## 自己尝试。
创建你自己的工具并在这里尝试！

In [ ]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """执行你最喜欢的运算

    Args:
        a (float): 操作数 a 描述
        b (float): 操作数 b 描述

    Returns:
        float: 描述
    """
    pass

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)